In [1]:
from tools import train_net_shapenet

class args:
    config_file = "configs/shapenet/voxmesh_R50.yaml"
    opts = ""
    num_gpus = 0
    data_dir = 'ShapeNetV1processed'
    copy_data = False
    no_color = True

In [2]:
cfg = train_net_shapenet.setup(args())
loaders = train_net_shapenet.setup_loaders(cfg)

[12/28 21:27:13] shapenet INFO: Using 0 GPUs per machine. Rank of current process: 0
[12/28 21:27:13] shapenet INFO: <__main__.args object at 0x7f8b540cefa0>
[12/28 21:27:14] shapenet INFO: Environment info:
----------------------  ---------------------------------------------------------------------
sys.platform            linux
Python                  3.9.12 (main, Apr  5 2022, 06:56:58) [GCC 7.5.0]
numpy                   1.21.5
detectron2              0.6 @/home/alan/miniconda3/lib/python3.9/site-packages/detectron2
Compiler                GCC 9.4
CUDA compiler           CUDA 11.7
detectron2 arch flags   8.6
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.12.1 @/home/alan/miniconda3/lib/python3.9/site-packages/torch
PyTorch debug build     False
GPU available           Yes
GPU 0                   NVIDIA RTX A5000 (arch=8.6)
Driver version          515.86.01
CUDA_HOME               /home/alan/miniconda3
Pillow                  9.2.0
torchvision             0.13.1 @/home/

[12/28 21:27:14] shapenet.data.mesh_vox INFO: Starting synset 03691459
[12/28 21:27:14] shapenet.data.mesh_vox INFO: Skipping synset 03691459
[12/28 21:27:14] shapenet.data.mesh_vox INFO: Starting synset 04090263
[12/28 21:27:14] shapenet.data.mesh_vox INFO: Skipping synset 04090263
[12/28 21:27:14] shapenet.data.mesh_vox INFO: Starting synset 04256520
[12/28 21:27:14] shapenet.data.mesh_vox INFO: Skipping synset 04256520
[12/28 21:27:14] shapenet.data.mesh_vox INFO: Starting synset 04379243
[12/28 21:27:14] shapenet.data.mesh_vox INFO: Skipping synset 04379243
[12/28 21:27:14] shapenet.data.mesh_vox INFO: Starting synset 04401088
[12/28 21:27:14] shapenet.data.mesh_vox INFO: Skipping synset 04401088
[12/28 21:27:14] shapenet.data.mesh_vox INFO: Starting synset 04530566
[12/28 21:27:14] shapenet.data.mesh_vox INFO: Skipping synset 04530566
[12/28 21:27:14] shapenet.data.build_data_loader INFO: Building dataset for split "val"
[12/28 21:27:14] shapenet.data.mesh_vox INFO: Starting synse

In [6]:
from plotly.subplots import make_subplots
import numpy as np
import plotly.graph_objects as go

for i, batch in enumerate(loaders["train"]):
    batch = loaders["train"].postprocess(batch, 'cpu')
    _, _, _, _, voxels_gt = batch
    print(loaders["train"].dataset.__len__())
    break
    fig = make_subplots(
    rows=6, cols=1,
    specs=[[{'type': 'volume'}],
           [{'type': 'volume'}],
           [{'type': 'volume'}],
           [{'type': 'volume'}],
           [{'type': 'volume'}],           
           [{'type': 'volume'}]])    
     
    for index in range(0, 6):
 
        X, Y, Z = np.mgrid[0:48:48j, 0:48:48j, 0:48:48j]

        fig.add_trace(
            go.Volume(
                x=X.flatten(),
                y=Y.flatten(),
                z=Z.flatten(),
                value=voxels_gt[index].detach().numpy().flatten(),
                isomin=0,
                isomax=1,
                opacity=0.1, # needs to be small to see through all surfaces
                surface_count=20, # needs to be a large number for good volume rendering
                ),
            row=index + 1,
            col=1,
        )   

        print(np.histogram(voxels_gt[index].detach().numpy().flatten()))
  
    fig.show()
     
    break

136680
